In [2]:
# importo librerias
    
import pandas as pd
import ast
from ast import literal_eval
import shutil
import gzip
import os

In [3]:
# cargo archivos

Games = pd.read_csv("/user/Documents/SANDRA 2023/HENRY/PROYECTO 1/DATA ETL/Games_ETL.csv")
Revision_usuarios = pd.read_csv("/user/Documents/SANDRA 2023/HENRY/PROYECTO 1/DATA ETL/Revision_usuarios_ETL.csv")


In [4]:
User_items = pd.read_csv("/user/Documents/SANDRA 2023/HENRY/PROYECTO 1/DATA ETL/User_Items_ETL.csv")

C:\Users\SANDRA\AppData\Local\Temp\ipykernel_15064\3884758258.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  User_items = pd.read_csv("/user/Documents/SANDRA 2023/HENRY/PROYECTO 1/DATA ETL/User_Items_ETL.csv")


1. Dataframe para Funcion para PlayTimeGenre

In [6]:
# Tomo del archivo las columnas que me interesan y las agrupo por item_name y sumo el tiempo juego

Juegos_horas = User_items[["item_name", "playtime_forever"]]
Juegos_horas = Juegos_horas.groupby(['item_name'], as_index=False)['playtime_forever'].sum()
Juegos_horas

,item_name,playtime_forever
0,神明的一天世界(God's One Day World),6969
1,! That Bastard Is Trying To Steal Our Gold !,742
2,"""Glow Ball"" - The billiard puzzle game",21
3,#SelfieTennis,261
4,#SkiJump,0
...,...,...
10942,丛林守望者（Ranger of the jungle）,2255
10943,侠客风云传(Tale of Wuxia),19288
10944,大海战 Navy Field IV,126
10945,神楽道中記(KaguraDouchuuki),4337


In [7]:
# Fusiono los DataFrames con columna en comun

df_1 = pd.merge(Games, Juegos_horas, left_on='Nombre_juego', right_on='item_name', how='inner')

df_1.drop_duplicates(inplace=True)

# Selecciono las columnas que me interesan
df_1 = df_1[["Genero", "Año_Lanzamiento", "playtime_forever"]]

# Convierto las cadenas en listas
df_1["Genero"] = df_1["Genero"].apply(ast.literal_eval)

# Descompongo las listas de 'Genero' y crear nuevas filas para cada elemento de la lista.
df_1 = df_1.explode('Genero')

df_1


,Genero,Año_Lanzamiento,playtime_forever
0,Action,1997,9319
0,Indie,1997,9319
0,Racing,1997,9319
1,Action,1998,2682852
2,Strategy,2006,27397
...,...,...,...
7109,Action,2003,758991
7110,Strategy,2003,3393
7110,RPG,2003,3393
7110,Indie,2003,3393


In [17]:
# Agrupar el DataFrame por las columnas 'Genero' y 'Año' y sumar 'playtime_forever'
df_1= df_1.groupby(['Genero', 'Año_Lanzamiento'], as_index=False)['playtime_forever'].sum()

df_1

,Genero,Año_Lanzamiento,playtime_forever
0,Action,1983,3582
1,Action,1984,384
2,Action,1988,16243
3,Action,1989,607
4,Action,1990,5
...,...,...,...
314,Web Publishing,2012,1947406
315,Web Publishing,2013,140060
316,Web Publishing,2015,332496
317,Web Publishing,2016,1


In [18]:
df_1.to_csv("/user/Documents/SANDRA 2023/HENRY/PROYECTO 1/DATA FUNCIONES/PlayTimeGenre.csv", index = False)

2. Dataframe para Funcion para UserForGenre

In [8]:
# Fusiono dataframes por columna en comun
df_2 = pd.merge(Games, User_items, left_on='Nombre_juego', right_on='item_name', how='inner')

df_2.drop_duplicates(inplace=True)

# Dejo las columnas que me interesan
df_2 = df_2[["Genero", "Año_Lanzamiento", "user_id", "playtime_forever"]]

# Evaluo y descompongo
df_2["Genero"] = df_2["Genero"].apply(ast.literal_eval)
df_2 = df_2.explode('Genero')


In [20]:
df_2

,Genero,Año_Lanzamiento,user_id,playtime_forever
0,Action,1997,1silentsniper1,0
0,Indie,1997,1silentsniper1,0
0,Racing,1997,1silentsniper1,0
1,Action,1997,420fedora,147
1,Indie,1997,420fedora,147
...,...,...,...,...
3065999,Action,2004,zowskimelon,232
3066000,Action,2004,zv_odd,0
3066001,Action,2004,zwanzigdrei,0
3066002,Action,2004,zyr0n1c,10


In [21]:
# Agrupo por las columnas 'Genero' y 'Año' y sumar 'playtime_forever'
df_2 = df_2.groupby(['Genero', 'Año_Lanzamiento', "user_id"], as_index=False)['playtime_forever'].sum()

# Renombro las columnas
df_2.rename(columns={'user_id': 'Id_User', 'playtime_forever': 'Horas_Jugadas'}, inplace=True)
df_2

,Genero,Año_Lanzamiento,Id_User,Horas_Jugadas
0,Action,1983,76561197997129113,40
1,Action,1983,76561198006449409,78
2,Action,1983,76561198007478325,47
3,Action,1983,76561198009435936,0
4,Action,1983,76561198025071276,41
...,...,...,...,...
2948441,Web Publishing,2017,Eosoforcus,58
2948442,Web Publishing,2017,N47H4NI3L,1635
2948443,Web Publishing,2017,dirklah,796
2948444,Web Publishing,2017,kushziller,251


In [23]:
df_2.to_csv("/user/Documents/SANDRA 2023/HENRY/PROYECTO 1/DATA FUNCIONES/UserForGenre.csv", index = False)

3. Dataframe para funcion UsersRecommend

In [25]:
#Selecciono columnas que me interesan y borro duplicados

Items_Caract = User_items.drop(columns={"user_id", "playtime_forever"}, axis=1)
Items_Caract= Items_Caract.drop_duplicates(subset='item_id')

In [27]:
# Fusiono por columna en comun y selecciono columnas que me interesan

df_union = pd.merge(Revision_usuarios, Items_Caract, on='item_id')
df_union = df_union[["recommend", "posted", "sentiment_analysis", "item_name"]]

df_union

,recommend,posted,sentiment_analysis,item_name
0,True,2011,2,Killing Floor
1,True,2015,2,Killing Floor
2,True,2013,0,Killing Floor
3,True,2015,1,Killing Floor
4,True,2014,1,Killing Floor
...,...,...,...,...
43397,True,2015,2,Aero's Quest
43398,True,2015,2,Aero's Quest
43399,False,2015,0,Another Perspective
43400,True,2015,2,The Howler


In [28]:
# Filtro por recomendacion y sentiment analysis mayor a cero y elimino columnas
df_3 = df_union[(df_union['recommend'] == True) & (df_union['sentiment_analysis'] > 0)]
df_3 = df_3.drop(columns={"recommend", "sentiment_analysis"}, axis=1)

# Agrupo por columnas y hago recuento en columna renombrada
df_3 = df_3.groupby(['posted', 'item_name']).size().reset_index(name='Cant_recomendaciones')
df_3.rename(columns={'posted': 'Año', 'item_name': 'Juego', 'Cant_recomendaciones': 'Recomendaciones' }, inplace=True)


In [29]:
df_3

,Año,Juego,Recomendaciones
0,2010,Amnesia: The Dark Descent,1
1,2010,ArcaniA,1
2,2010,Audiosurf,1
3,2010,Chime,1
4,2010,Counter-Strike: Source,2
...,...,...,...
3948,2015,resident evil 4 / biohazard 4,1
3949,2015,sZone-Online,2
3950,2015,the static speaks my name,9
3951,2015,theHunter,13


In [30]:
df_3.to_csv("/user/Documents/SANDRA 2023/HENRY/PROYECTO 1/DATA FUNCIONES/UsersRecommend.csv" , index = False)

4. Dataframe para Funcion UsersNotRecommend

In [33]:
# Filtra por recomendacion y Sentiment analysis igual a cero y elimino columnas
df_4 = df_union[(df_union['recommend'] == False) & (df_union['sentiment_analysis'] == 0)]
df_4 = df_4.drop(columns={"recommend", "sentiment_analysis"}, axis=1)

# Agrupo y cuento numero de filas, renombro columnas
df_4 = df_4.groupby(['posted', 'item_name']).size().reset_index(name='Recomendaciones')
df_4.rename(columns={'posted': 'Año', 'item_name': 'Juego'}, inplace=True)


In [34]:
df_4

,Año,Juego,Recomendaciones
0,2011,And Yet It Moves,2
1,2011,Dead Island,1
2,2011,Fable III,1
3,2011,Men of War: Vietnam,1
4,2012,Call of Duty: Black Ops II,1
...,...,...,...
936,2015,Zombie Army Trilogy,2
937,2015,Zombie Zoeds,1
938,2015,bit Dungeon II,1
939,2015,sZone-Online,1


In [35]:
df_4.to_csv("/user/Documents/SANDRA 2023/HENRY/PROYECTO 1/DATA FUNCIONES/UsersNoRecommend.csv" , index = False)

5. Dataframe para funcion Sentiment_analysis

In [36]:
# Fusiono las columnas que me interesan y dejo las colummnas que me interesan

df_5 = pd.merge(df_union, Games, left_on='item_name', right_on='Nombre_juego', how='inner')
df_5 = df_5[["Año_Lanzamiento", "sentiment_analysis"]]

#Agrupo, cuento filas y reseteo el indice para crear la columna nueva 
df_5 = df_5.groupby(['Año_Lanzamiento', 'sentiment_analysis']).size().reset_index(name='Registros')


In [37]:
df_5

,Año_Lanzamiento,sentiment_analysis,Registros
0,1989,2,1
1,1991,2,1
2,1992,2,2
3,1993,0,1
4,1993,2,2
...,...,...,...
69,2016,1,134
70,2016,2,558
71,2017,0,259
72,2017,1,335


In [38]:
df_5.to_csv("/user/Documents/SANDRA 2023/HENRY/PROYECTO 1/DATA FUNCIONES/Sentiment_Analysis.csv" , index = False)